In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x84600e0


In [2]:
n = 2
n_bins = 100
n_events_data = 5000000
n_events_mc = n_events_data * 500

x = []
for i in range(n):
    var = ROOT.RooRealVar(f"x_{i}", f"x_{i}", 0., -3., 3.)
    var.setBins(n_bins)
    x.append(var)

In [3]:
cov_mat1 = ROOT.TMatrixD(ROOT.TMatrixD.kUnit, ROOT.TMatrixD(n, n))
cov_mat1[0][1] = 0.42
cov_mat1[1][0] = 0.42
gauss1 = ROOT.RooMultiVarGaussian("gauss1", "gauss1", ROOT.RooArgList(x), cov_mat1)

cov_mat2 = ROOT.TMatrixD(ROOT.TMatrixD.kUnit, ROOT.TMatrixD(n, n))
cov_mat2[0][1] = 0.1337
cov_mat2[1][0] = 0.1337
gauss2 = ROOT.RooMultiVarGaussian("gauss2", "gauss2", ROOT.RooArgList(x), cov_mat2)

In [4]:
# our mixing angle
theta = ROOT.RooRealVar("theta", "theta", 0.5, 0., 1.)

gen_model = ROOT.RooAddPdf("gen_model", "gen_model", gauss1, gauss2, theta)

In [5]:
gen_model.Print("t")

0xab757e0 RooAddPdf::gen_model = 1/1 [Auto,Clean] 
  0xab0d490/V- RooMultiVarGaussian::gauss1 = 1 [Auto,Dirty] 
    0xa2450f0/V- RooRealVar::x_0 = 0
    0x8439120/V- RooRealVar::x_1 = 0
    0xaaefab0/V- RooConstVar::0 = 0
  0xa9c3490/V- RooMultiVarGaussian::gauss2 = 1 [Auto,Dirty] 
    0xa2450f0/V- RooRealVar::x_0 = 0
    0x8439120/V- RooRealVar::x_1 = 0
    0xaaefab0/V- RooConstVar::0 = 0
  0x89801f0/V- RooRealVar::theta = 0.5


In [6]:
# data = gen_model.generate(x, 1)
data = gen_model.generateBinned(x, n_events_data)
# template_data = gen_model.generateBinned(x, n_events_mc)
data.Print("v")

DataStore genData (genData)
  Contains 10000 entries
  Observables: 
    1)  x_0 = 2.97  L(-3 - 3)  "x_0"
    2)  x_1 = 2.97  L(-3 - 3)  "x_1"
Binned Dataset genData (genData)
  Contains 10000 bins with a total weight of 5e+06
  Observables:     1)  x_0 = 2.97  L(-3 - 3)  "x_0"
    2)  x_1 = 2.97  L(-3 - 3)  "x_1"


In [7]:
# plot generated data
frame0 = x[0].frame()
frame1 = x[1].frame()

In [8]:
data.plotOn(frame0)
gen_model.plotOn(frame0)
data.plotOn(frame1)
gen_model.plotOn(frame1)

[#1] INFO:Plotting -- RooAbsReal::plotOn(gen_model) plot on x_0 integrates over variables (x_1)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(gen_model_Int[x_1]_Norm[x_0,x_1]) using numeric integrator RooIntegrator1D to calculate Int(x_1)
[#1] INFO:Plotting -- RooAbsReal::plotOn(gen_model) plot on x_1 integrates over variables (x_0)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(gen_model_Int[x_0]_Norm[x_0,x_1]) using numeric integrator RooIntegrator1D to calculate Int(x_0)


In [9]:
c0 = ROOT.TCanvas()
frame0.Draw()
c0.Draw()

c1 = ROOT.TCanvas()
frame1.Draw()
c1.Draw()

In [10]:
nll = gen_model.createNLL(data)

[#1] INFO:Fitting -- RooAbsPdf::fitTo(gen_model) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- Creation of NLL object took 26.6716 ms


In [11]:
nll_minimizer = ROOT.RooMinimizer(nll)

[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_gen_model_genData) Summation contains a RooNLLVar, using its error level


In [ ]:
# obviously cannot fit theta as the relevant information is lost in the projection
# and there is no element to recover it added to the fit
# I could space the gaussians a bit but then it becomes very trivial?
nll_minimizer.migrad()

0

Minuit2Minimizer: Minimize with max-calls 500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 13856498.6227606665
Edm   = 2.67200619875268504e-09
Nfcn  = 17
theta	  = 0.502079	 +/-  0.00144896	(limited)


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 2.06377 ms
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       13856499.65 Edm =       1.027953668 NCalls =      7
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 13856499.65
  Edm           : 1.027953668
  Internal parameters:	[                0]	
  Internal gradient  :	[     -494.3862711]	
  Internal covariance matrix:
[[  1.6822895e-05]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       13856499.65 Edm =       1.027953668 NCalls =      7
Info in <Minuit2>: VariableMetricBuilder    1 - FCN =       13856498.62 Edm =   6.328999198e-07 NCalls =     10
Info in <Minuit2>: MnHesse Done after 397.764 μs
Info in <Minuit2>: VariableMetricBuilder After Hessian
Info in <Minuit2>: VariableMetricBuilder  

In [13]:
frame_theta = theta.frame()
nll.plotOn(frame_theta, ShiftToZero=True)
c_theta = ROOT.TCanvas()
frame_theta.Draw()
c_theta.Draw()

In [14]:
# naive fit model 1
# make 1d histos from template
gauss1_tmpl = gauss1.generateBinned(x, n_events_mc)
gauss2_tmpl = gauss2.generateBinned(x, n_events_mc)

gauss1_tmpl_0 = gauss1_tmpl.reduce(x[0])
gauss1_tmpl_1 = gauss1_tmpl.reduce(x[1])
gauss2_tmpl_0 = gauss2_tmpl.reduce(x[0])
gauss2_tmpl_1 = gauss2_tmpl.reduce(x[1])

[#0] PROGRESS:Generation -- RooAbsPdf::generateBinned(gauss1) Performing costly accept/reject sampling. If this takes too long, use extended mode to speed up the process.
[#0] PROGRESS:Generation -- RooAbsPdf::generateBinned(gauss2) Performing costly accept/reject sampling. If this takes too long, use extended mode to speed up the process.


In [15]:
gauss1_tmpl_0_hf = ROOT.RooHistFunc("gauss1_tmpl_0_hf", "gauss1_tmpl_0_hf", x[0], gauss1_tmpl_0)
gauss1_tmpl_1_hf = ROOT.RooHistFunc("gauss1_tmpl_1_hf", "gauss1_tmpl_1_hf", x[1], gauss1_tmpl_1)
gauss2_tmpl_0_hf = ROOT.RooHistFunc("gauss2_tmpl_0_hf", "gauss2_tmpl_0_hf", x[0], gauss2_tmpl_0)
gauss2_tmpl_1_hf = ROOT.RooHistFunc("gauss2_tmpl_1_hf", "gauss2_tmpl_1_hf", x[1], gauss2_tmpl_1)

In [16]:
fit_theta = ROOT.RooRealVar("fit_theta", "fit_theta", 0.5, 0., 1.)

sum_tmpl_0 = ROOT.RooRealSumPdf("sum_tmpl_0", "sum_tmpl_0", gauss1_tmpl_0_hf, gauss2_tmpl_0_hf, fit_theta)
sum_tmpl_1 = ROOT.RooRealSumPdf("sum_tmpl_1", "sum_tmpl_1", gauss1_tmpl_1_hf, gauss2_tmpl_1_hf, fit_theta)

In [17]:
fit_model = ROOT.RooProdPdf("fit_model", "fit_model", sum_tmpl_0, sum_tmpl_1)
fit_model.Print("t")

0xaf16f10 RooProdPdf::fit_model = 4.60473e+15 [Auto,Dirty] 
RooProdPdf begin partial integral cache
[0]0xc2349d0 RooRealSumPdf::sum_tmpl_0 = 6.78585e+07 [Auto,Dirty] 
[0]  0xc268410/V- RooHistFunc::gauss1_tmpl_0_hf = 6.79215e+07 [Auto,Clean] 
[0]    0xa2450f0/V- RooRealVar::x_0 = 0
[0]  0xc1862f0/V- RooHistFunc::gauss2_tmpl_0_hf = 6.77956e+07 [Auto,Clean] 
[0]    0xa2450f0/V- RooRealVar::x_0 = 0
[0]  0xc408250/V- RooRealVar::fit_theta = 0.5
[0]0xa5122b0 RooRealSumPdf::sum_tmpl_1 = 6.78578e+07 [Auto,Dirty] 
[0]  0xc1d19e0/V- RooHistFunc::gauss1_tmpl_1_hf = 6.79241e+07 [Auto,Clean] 
[0]    0x8439120/V- RooRealVar::x_1 = 0
[0]  0xc408fe0/V- RooHistFunc::gauss2_tmpl_1_hf = 6.77914e+07 [Auto,Clean] 
[0]    0x8439120/V- RooRealVar::x_1 = 0
[0]  0xc408250/V- RooRealVar::fit_theta = 0.5
RooProdPdf end partial integral cache
  0xc2349d0/V- RooRealSumPdf::sum_tmpl_0 = 6.78585e+07 [Auto,Dirty] 
    0xc268410/V- RooHistFunc::gauss1_tmpl_0_hf = 6.79215e+07 [Auto,Clean] 
      0xa2450f0/V- RooRealVa

In [18]:
fit_model.fitTo(data)

<cppyy.gbl.RooFitResult object at 0x(nil)>


[#1] INFO:Fitting -- RooAbsPdf::fitTo(fit_model) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- Creation of NLL object took 452.534 μs
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_fit_model_genData) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 14021741.7183042523
Edm   = 3.35676930375630703e-08
Nfcn  = 30
fit_theta	  = 0.319685	 +/-  0.124807	(limited)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: deactivating const optimization


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 827.256 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       14021742.82 Edm =       1.096812688 NCalls =      3
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 14021742.82
  Edm           : 1.096812688
  Internal parameters:	[                0]	
  Internal gradient  :	[      6.051386004]	
  Internal covariance matrix:
[[     0.11980715]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       14021742.82 Edm =       1.096812688 NCalls =      3
Info in <Minuit2>: VariableMetricBuilder    1 - FCN =       14021741.72 Edm =     0.01694219564 NCalls =      6
Info in <Minuit2>: VariableMetricBuilder    2 - FCN =       14021741.72 Edm =     0.01537910272 NCalls =     12
Warning in <Minuit2>: VariableMetr